In [1]:
import pymongo
import datetime

ModuleNotFoundError: No module named 'pymongo'

In [2]:
# connect to localhost on default port 27017
client = pymongo.MongoClient('127.0.0.1', 27017)

In [3]:
# get/create a database with name `database_name`
db = client['Punks']

In [4]:
# get/create a collection in the database called `test_col`
col = db.get_collection('punks')

In [5]:
col

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'Punks'), 'punks')

In [6]:
# json sample for local testing

import json 
with open('sample-assignment2.json') as f:
    data = json.load(f)
    
# col.insert_many(data)

In [7]:
len(data[5]['total_price'])

20

## Question 1

In [8]:
import requests

# url = "https://api.opensea.io/api/v1/events?only_opensea=false&offset=0&limit=20"
url = "https://us-ie-big-data-technologies.ew.r.appspot.com/api/v1/events?offset=0&limit=10000&occurred_after=2021-09-01T00%3A00%3A00%2B02%3A00&occurred_before=2021-10-01T00%3A00%3A00%2B02%3A00&collection_slug=cryptopunks"

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

response_json_data = response.json()
col.insert_many(response_json_data["asset_events"])

## Question 2

In [9]:
db.punks.count()

/home/f5047145/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


534

## Question 3

In [10]:
print(list(db.punks.aggregate( [
    {
       "$project":
          {
            "asset.token_id": 1,
            "total_price": {"$divide": [{"$convert":{
                "input":{
                    "$substr": [ "$total_price", 0, 4]
                }, "to":"double"}
                                        },10]},
          }
      },
    {"$match":{}},
    {"$sort":{"total_price":-1}},
    {"$limit":10}
                ])
          ))

[{'_id': ObjectId('61b4c1020e646336f20012f8'), 'asset': {'token_id': '3634'}, 'total_price': 999.9}, {'_id': ObjectId('61b4c1020e646336f2001402'), 'asset': {'token_id': '8922'}, 'total_price': 999.9}, {'_id': ObjectId('61b4c1020e646336f20013db'), 'asset': {'token_id': '7633'}, 'total_price': 999.8}, {'_id': ObjectId('61b4c1020e646336f200141a'), 'asset': {'token_id': '9103'}, 'total_price': 999.0}, {'_id': ObjectId('61b4c1020e646336f2001299'), 'asset': {'token_id': '1672'}, 'total_price': 997.5}, {'_id': ObjectId('61b4c1020e646336f20012ca'), 'asset': {'token_id': '3239'}, 'total_price': 996.9}, {'_id': ObjectId('61b4c1020e646336f20012df'), 'asset': {'token_id': '3533'}, 'total_price': 995.0}, {'_id': ObjectId('61b4c1020e646336f20012d8'), 'asset': {'token_id': '3378'}, 'total_price': 995.0}, {'_id': ObjectId('61b4c1020e646336f20012c3'), 'asset': {'token_id': '3173'}, 'total_price': 990.0}, {'_id': ObjectId('61b4c1020e646336f2001303'), 'asset': {'token_id': '4610'}, 'total_price': 990.0}]

## Question 4

In [11]:
len(db.punks.find({"event_type":"successful",'asset.asset_contract.schema_name':'CRYPTOPUNKS'}).distinct("asset.name"))

395

## Question 5

In [12]:
results = db.punks.aggregate( [ {
   "$project": { 
      "event_type":1,
      "total_price":1,
      "transaction.timestamp": {
         "$dateFromString": {
            "dateString": '$transaction.timestamp',
            "timezone": 'GMT',
            "onError": "transaction.timestamp"
         }
      }
   }
},
{"$match" : { "event_type": "successful"}},
{"$group": { 
        "_id": {
               "day": {"$dayOfMonth": "$transaction.timestamp"},
               "month": {"$month": "$transaction.timestamp"}, 
               "year": {"$year": "$transaction.timestamp"}
             },
        "Total between time period": {
            "$sum" : 1
        }}
},
{"$sort":{"Total between time period":-1}},
{"$limit":1}])
print(list(results))

[{'_id': {'day': 16, 'month': 9, 'year': 2021}, 'Total between time period': 90}]


## Question 6

In [13]:
print(list(db.punks.aggregate( [
    {
       "$project":
          {
            "event_type": 1,
            "payment_token.symbol": 1,
            "total_price": {"$divide": [{"$convert":{
                "input":{
                    "$substr": [ "$total_price", 0,  4]
                }, "to":"double"}
                                        },10]},
           "transaction.timestamp": {
                     "$dateFromString": {
                                        "dateString": '$transaction.timestamp',
                                        "timezone": 'GMT',
                                        "onError": "transaction.timestamp"
                                         }
                                      }
          }
      },
   { "$match": { "event_type": "successful",
                
                "transaction.timestamp":{"$gte":datetime.datetime(2021, 9, 16), 
                                      "$lt": datetime.datetime(2021, 9, 17)}
               } },
    {"$group": { 
        "_id": "payment_token.symbol.ETH",
        "Total sum in ETH": {
            "$sum": "$total_price"
        }
}}])))

[{'_id': 'payment_token.symbol.ETH', 'Total sum in ETH': 38580.8}]
